In [ ]:
# 필요한 파이썬 모듈 설치 - 모듈 설치가 전부 되어 있다면 실행하지 않기
from IPython.display import clear_output
! pip install tqdm
! pip install sentence_transformers
! pip install pandas
clear_output()
print("설치된 모듈 : tqdm, sentence_transformers")

In [1]:
# SentenceBERT 모델 종류 선언
from sentence_transformers import SentenceTransformer

model_type = [
    # 기본 SentenceBERT 모델 (0~1)
    "bert-large-nli-mean-tokens",
    "bert-large-nli-stsb-mean-tokens",
    # 사전 훈련된 SentenceBERT 모델 (2~3)
    "distilbert-base-nli-stsb-mean-tokens", # 가볍고 빠름
    "roberta-large-nli-stsb-mean-tokens", # 무겁지만 성능이 뛰어남
    # 다국어 SentenceBERT 모델 (4~5) - 한국어 포함
    "paraphrase-multilingual-MiniLM-L12-v2",
    "distiluse-base-multilingual-cased",
    # 도메인 특화 SBERT 모델 (6~7)
    "all-mpnet-base-v2", # 일반적인 문장 의미에 좋은 성능을 보여줌
    "msmarco-distilbert-base-v4", # 검색 관련 좋은 성능을 보여줌
    # 한국어 SBERT 모델 (8~9)
    "BM-K/KoSimCSE-roberta", # 다양한 한국어 데이터로 학습된 모델
    "jhgan/ko-sbert-sts", # 한국어 문장 유사도 (STS) 데이터로 학습한 모델
]

In [2]:
# 모델 성능 테스트
from sentence_transformers import util
from scipy.stats import spearmanr
from tqdm import tqdm
import pandas as pd
import time, os, torch

device = torch.device("cpu")

# STS 데이터셋 불러오기
# sts_type = "STSB"
# sts_list = os.listdir(f"./data/{sts_type}")
# sts_list = [f for f in sts_list if f[0] != '.']
# if sts_list[0].endswith('.tsv'):
#     sts_df = pd.read_csv(f"./data/{sts_type}/" + sts_list[0], sep="\t", on_bad_lines='skip')
#     for sts_file in sts_list[1:]:
#         sts_df = pd.concat([sts_df.copy(), pd.read_csv(f"./data/{sts_type}/" + sts_file, sep='\t', on_bad_lines='skip')], ignore_index=True)
# elif sts_list[0].endswith('.parquet'):
#     sts_df = pd.read_parquet(f"./data/{sts_type}/" + sts_list[0], engine="pyarrow")
#     for sts_file in sts_list[1:]:
#         sts_df = pd.concat([sts_df.copy(), pd.read_parquet(f"./data/{sts_type}/" + sts_file, engine="pyarrow")], ignore_index=True)
sts_df = pd.read_csv(f"./data/KorSTS/sts-test.tsv", sep="\t", on_bad_lines='skip')
print(f"데이터 개수 : {len(sts_df)}")
sentence1 = sts_df["sentence1"].astype(str)
sentence2 = sts_df["sentence2"].astype(str)
score = sts_df["score"]

results = {}

for model_name in model_type[0:]:
    start_time = time.time()
    model = SentenceTransformer(model_name)
    embeddings1 = model.encode(sentence1, convert_to_tensor=True).to(device)
    embeddings2 = model.encode(sentence2, convert_to_tensor=True).to(device)
    cos_sim = torch.nn.functional.cosine_similarity(embeddings1, embeddings2).cpu().numpy()

    norm_cos_sim = (cos_sim + 1) * 2.5 # -1~1 범위를 0~5 범위로 정규화
    spearman_corr, _ = spearmanr(score, norm_cos_sim)

    results[model_name] = [spearman_corr, time.time() - start_time]
    torch.cuda.empty_cache()
    del model, embeddings1, embeddings2, cos_sim, norm_cos_sim, spearman_corr, _
    print(f"{model_name} : {results[model_name][0] : .4f}, {results[model_name][1] : .4f}s")

데이터 개수 : 1379
bert-large-nli-mean-tokens :  0.3686,  69.2232s
bert-large-nli-stsb-mean-tokens :  0.3241,  66.3417s
distilbert-base-nli-stsb-mean-tokens :  0.3651,  13.2555s
roberta-large-nli-stsb-mean-tokens :  0.3914,  73.2906s
paraphrase-multilingual-MiniLM-L12-v2 :  0.7392,  9.0851s
distiluse-base-multilingual-cased :  0.7421,  9.2493s
all-mpnet-base-v2 :  0.3689,  23.3165s
msmarco-distilbert-base-v4 :  0.3319,  13.6524s


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


BM-K/KoSimCSE-roberta :  0.8009,  10.7297s
jhgan/ko-sbert-sts :  0.8078,  12.9897s


In [3]:
for model_name in model_type:
    print(f"{model_name} : {results[model_name][0] : .4f}, {results[model_name][1] : .4f}s")

bert-large-nli-mean-tokens :  0.8015,  54.7349s
bert-large-nli-stsb-mean-tokens :  0.9501,  156.1642s
distilbert-base-nli-stsb-mean-tokens :  0.9481,  32.3018s
roberta-large-nli-stsb-mean-tokens :  0.9447,  179.9490s
paraphrase-multilingual-MiniLM-L12-v2 :  0.8488,  25.7859s
distiluse-base-multilingual-cased :  0.7979,  34.7029s
all-mpnet-base-v2 :  0.8490,  61.0560s
msmarco-distilbert-base-v4 :  0.7749,  32.2363s
BM-K/KoSimCSE-roberta :  0.7139,  79.9904s
jhgan/ko-sbert-sts :  0.7401,  81.9747s


In [4]:
torch.cuda.empty_cache()

In [47]:
# 데이터 입력
import os

data_path = "./data/kwiki"
data_name = "kwiki_02"
os.makedirs(data_path, exist_ok=True)

with open(f"{data_path}/{data_name}.txt", "r", encoding="utf-8") as f:
    data_a = f.read() # 원본 문서
with open(f"{data_path}/{data_name}_q.txt", "r", encoding="utf-8") as f:
    data_b = f.read() # 질의 문서

In [23]:
# 1. 문서 전체 임베딩 알고리즘
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer(model_type[9])
print(f"Model Type : {model_type[9]}")

embedding_a = model.encode(data_a)
questions = data_b.split('\n')
for question in questions:
    embedding_b = model.encode(question)
    print(f"Q : {question}")
    print(f"S : {util.cos_sim(embedding_a, embedding_b).item() : .4f}")

Model Type : jhgan/ko-sbert-sts
Q : 일반 상대성이론은 1915년에 발표된 알베르트 아인슈타인의 이론으로, 뉴턴의 만유인력 법칙을 수정하였어.
S :  0.8384
Q : 우주에 관한 해를 얻기 위해서는 공간의 밀도가 균일한 먼지로 가득 채워져 있다는 전제 하에 아인슈타인 방정식을 대입해 알아낼 수 있어.
S :  0.6013
Q : 천체물리학에서 블랙혹이라는 밀도가 매우 높은 새로운 종류의 천체를 일반 상대론을 통해 예측합니다.
S :  0.5179


In [35]:
# 2. 문장 단위 임베딩 알고리즘
from sentence_transformers import SentenceTransformer, util
import re

model = SentenceTransformer(model_type[9])
print(f"Model Type : {model_type[9]}")

data_a = re.sub(r'\s*formula_\d+\s*', '', data_a)
data_a = re.sub(r'\n+', '\n', data_a).strip()

sentences_a = data_a.split('.'); sentences_a = [s + '.' for  s in sentences_a]
sentences_a = [s for s in sentences_a if len(s) >= 11]
questions = data_b.split('\n')

for question in questions:
    embedding_b = model.encode(question)
    max_sim = -1; max_idx = -1
    for idx in range(len(sentences_a)):
        cur_sim = util.cos_sim(model.encode(sentences_a[idx]), embedding_b)
        max_idx = idx if cur_sim > max_sim else max_idx
        max_sim = cur_sim if cur_sim > max_sim else max_sim
    print(f"Q : {question}")
    print(f"A : {sentences_a[max_idx]}")
    print(f"S : {max_sim.item() : .4f}")

Model Type : jhgan/ko-sbert-sts
Q : 관리자는 접속 기간 중 최소 80% 이상 보고서를 작성해야 합니다. 개인적인 이유로 결석이 필요할 경우, 반드시 사전에 본부장의 승인을 받아야 하며, 3일 이상 무단 결석 시 연수비 지원이 중단됩니다.
A :  관리자는 반드시 보고서를 접속 기간의 80% 이상 작성해야 한다.
S :  0.7545
Q : 관리가 종료된 후 7일 이내에 성과 보고서, 출석 기록, 그리고 계좌 사본을 각각 1부씩 제출해야 합니다.
A :  관리 종료 후 7일 이내 결과보고서, 출석부 및 계좌 사본 각 1부
 제출
 3.
S :  0.9208
Q : 관리자는 관리자 교육 과정을 필수적으로 이수해야 합니다.
A :  관리자는 반드시 관리자 교육 이수해야 한다.
S :  0.9182
Q : * 관리자 유의 사항 * 접속 기간 중 보고서 80% 이상 작성 필요
A :  관리자는 반드시 보고서를 접속 기간의 80% 이상 작성해야 한다.
S :  0.8883


In [48]:
# 3. 슬라이딩 윈도우 임베딩 알고리즘
from sentence_transformers import SentenceTransformer, util
import re

model = SentenceTransformer(model_type[9])
print(f"Model Type : {model_type[9]}")

data_a = re.sub(r'\s*formula_\d+\s*', '', data_a)
data_a = re.sub(r'\n+', '', data_a).strip()

sentences_a = []
window_size = 100 # Window Size

for i in range(0, len(data_a)-window_size):
    sentences_a.append(data_a[i:i+window_size])
questions = data_b.split('\n')

for question in questions:
    embedding_b = model.encode(question)
    max_sim = -1; max_idx = -1
    embedding_a = []
    for idx in range(len(sentences_a)):
        embedding_a.append(model.encode(sentences_a[idx]))
    for idx in range(len(sentences_a)):
        cur_sim = util.cos_sim(embedding_a[idx], embedding_b)
        max_idx = idx if cur_sim > max_sim else max_idx
        max_sim = cur_sim if cur_sim > max_sim else max_sim
    print(f"Q : {question}")
    print(f"A : {sentences_a[max_idx]}")
    print(f"S : {max_sim.item() : .4f}")

Model Type : jhgan/ko-sbert-sts
Q : 일반 상대성이론은 1915년에 발표된 알베르트 아인슈타인의 이론으로, 뉴턴의 만유인력 법칙을 수정하였어.
A : 은 1915년 발표된 알베르트 아인슈타인의 고전적 중력이론으로, 특수 상대론을 확장한 기하학적 중력 모형에 근거하여 뉴턴의 만유인력 법칙을 수정한 이론이다. 일반 상대론은 현대의 
S :  0.9148
Q : 우주에 관한 해를 얻기 위해서는 공간의 밀도가 균일한 먼지로 가득 채워져 있다는 전제 하에 아인슈타인 방정식을 대입해 알아낼 수 있어.
A : 물리 우주론.공간이 밀도가 균일한 먼지로 가득 채워져 있다고 가정하여 아인슈타인 방정식에 대입하면 우주에 관한 해를 얻을 수 있으며, 우주와 관련된 여러 지표에 따라서 우주 공간의
S :  0.9311
Q : 천체물리학에서 블랙혹이라는 밀도가 매우 높은 새로운 종류의 천체를 일반 상대론을 통해 예측합니다.
A :  천체물리학과 우주론의 기반이 된다. 천체물리학에서 일반 상대론은 중성자별, 블랙홀이라는 밀도가 매우 높아 극한의 중력 환경을 제공하는 새로운 종류의 천체를 예측한다. 이러한 천체
S :  0.7799
